In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/kaggle/input/titanic/train.csv')
dftest = pd.read_csv('/kaggle/input/titanic/test.csv')
extra = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
df.drop(['Name','Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df = pd.merge(df, pd.get_dummies(df, columns=['Sex']))

In [4]:
df['Target'] = df['Survived']
df.drop('Survived', axis=1, inplace=True)

df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Target
0,1,3,male,22.0,1,0,7.2500,0,1,0
1,2,1,female,38.0,1,0,71.2833,1,0,1
2,3,3,female,26.0,0,0,7.9250,1,0,1
3,4,1,female,35.0,1,0,53.1000,1,0,1
4,5,3,male,35.0,0,0,8.0500,0,1,0
...,...,...,...,...,...,...,...,...,...,...
886,887,2,male,27.0,0,0,13.0000,0,1,0
887,888,1,female,19.0,0,0,30.0000,1,0,1
888,889,3,female,NaN,1,2,23.4500,1,0,0
889,890,1,male,26.0,0,0,30.0000,0,1,1


In [5]:
df['Age'] = df['Age'].fillna(np.mean(df['Age']))
df.drop('Sex', axis=1,inplace=True)

In [6]:
X = df.iloc[:,:8]

y = df.iloc[:,8]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
X_train

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
105,106,3,28.000000,0,0,7.8958,0,1
68,69,3,17.000000,4,2,7.9250,1,0
253,254,3,30.000000,1,0,16.1000,0,1
320,321,3,22.000000,0,0,7.2500,0,1
706,707,2,45.000000,0,0,13.5000,1,0
...,...,...,...,...,...,...,...,...
835,836,1,39.000000,1,1,83.1583,1,0
192,193,3,19.000000,1,0,7.8542,1,0
629,630,3,29.699118,0,0,7.7333,0,1
559,560,3,36.000000,1,0,17.4000,1,0


In [9]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=30)
clf.fit(X_train, y_train)



RandomForestClassifier(max_depth=30)

In [10]:
clf.score(X_test, y_test)

0.8385650224215246

In [11]:
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(n_estimators = 64, max_depth = 32)

grid_values = {'max_depth': [1, 2, 4, 8, 16, 32, 64, 100, 200]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train, y_train)


GridSearchCV(estimator=RandomForestClassifier(max_depth=32, n_estimators=64),
             param_grid={'max_depth': [1, 2, 4, 8, 16, 32, 64, 100, 200]})

In [12]:
grid_clf_acc.cv_results_
grid_clf_acc.best_estimator_

RandomForestClassifier(max_depth=200, n_estimators=64)

In [13]:
clf = RandomForestClassifier(n_estimators = 64, max_depth = 32)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))



0.8475336322869955


In [14]:
dftest

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [15]:
dftest = pd.read_csv('/kaggle/input/titanic/test.csv')

dftest.drop(['Name','Ticket','Cabin','Embarked'], axis=1, inplace=True)
dftest = pd.merge(dftest, pd.get_dummies(dftest, columns=['Sex']))
dftest['Age'] = dftest['Age'].fillna(np.mean(dftest['Age']))
dftest.drop('Sex', axis=1,inplace=True)

In [16]:
dftest

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,892,3,34.50000,0,0,7.8292,0,1
1,893,3,47.00000,1,0,7.0000,1,0
2,894,2,62.00000,0,0,9.6875,0,1
3,895,3,27.00000,0,0,8.6625,0,1
4,896,3,22.00000,1,1,12.2875,1,0
...,...,...,...,...,...,...,...,...
413,1305,3,30.27259,0,0,8.0500,0,1
414,1306,1,39.00000,0,0,108.9000,1,0
415,1307,3,38.50000,0,0,7.2500,0,1
416,1308,3,30.27259,0,0,8.0500,0,1


In [17]:
pd.set_option('display.max_rows', 1000)

In [18]:
dftest['Fare'] = dftest['Fare'].fillna(np.mean(dftest['Fare']))



In [19]:
result = pd.DataFrame(dftest['PassengerId'])
result['Survived'] = clf.predict(dftest)

In [20]:
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,0
9,901,0


In [21]:
result.to_csv('submission.csv', index=False)

In [22]:
clf = RandomForestClassifier()

grid_values = {'max_depth': [1, 2, 4, 8, 16, 32, 64, 100, 200], 'n_estimators':[1, 2, 4, 8, 16, 32, 64, 100, 200]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train, y_train)
grid_clf_acc.cv_results_


{'mean_fit_time': array([0.00533156, 0.0072947 , 0.01112309, 0.01869864, 0.03384061,
        0.06423426, 0.12482328, 0.19165087, 0.38102779, 0.00643601,
        0.00804448, 0.01090193, 0.0189446 , 0.03422022, 0.06710196,
        0.12773609, 0.19630742, 0.3902288 , 0.0053185 , 0.00712805,
        0.01133962, 0.01940846, 0.03602452, 0.0678165 , 0.13241224,
        0.20264883, 0.40143714, 0.00537248, 0.00739188, 0.01166792,
        0.0204464 , 0.03747087, 0.07234836, 0.14331417, 0.21638584,
        0.42425108, 0.00535617, 0.00750432, 0.01204762, 0.02088456,
        0.03805943, 0.07297187, 0.14246612, 0.22235484, 0.43972406,
        0.00540867, 0.00767837, 0.01207142, 0.02098899, 0.03869348,
        0.07446451, 0.1435966 , 0.22997146, 0.43980522, 0.00551729,
        0.00774822, 0.01239786, 0.02598376, 0.04352694, 0.07690544,
        0.14604259, 0.22228317, 0.43894529, 0.00558763, 0.00755424,
        0.01210485, 0.02093   , 0.03844271, 0.08198929, 0.14349837,
        0.22953296, 0.44778709,

In [23]:
grid_clf_acc.best_estimator_


RandomForestClassifier(max_depth=8, n_estimators=200)

In [24]:
clf = RandomForestClassifier(max_depth=8, n_estimators=64)
clf.fit(X,y)


RandomForestClassifier(max_depth=8, n_estimators=64)

In [25]:
results2=pd.DataFrame({'PassengerId':dftest['PassengerId'], 'Survived':clf.predict(dftest)})

results2

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [26]:
results2.to_csv('submission.csv', index=False)